In [28]:

from collections import Counter
import numpy as np


import csv
import pandas as pd

In [29]:
import requests
#    url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
url = 'https://www.reddit.com/r/wallstreetbets/search?q=flair_name%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
requests.get(url)

<Response [200]>

In [30]:
html = requests.get(url)   
print(html.text)

"enabled","version":"6"},"ios_econ_prediction_sheet_header":{"id":1659,"name":"ios_econ_prediction_sheet_header","variant":"enabled","version":"2"},"ios_gql_badges_v2":{"id":672,"name":"ios_gql_badges_v2","variant":"enabled","version":"13"},"ios_location_recommendations_toggle":{"id":966,"name":"ios_location_recommendations_toggle","variant":"enabled","version":"3"},"ios_new_community_progress":{"id":1518,"name":"ios_new_community_progress","variant":"enabled","version":"2"},"ios_rpan_award_presence":{"id":764,"name":"ios_rpan_award_presence","variant":"enabled","version":"3"},"ios_rpan_broadcaster_award_presence":{"id":873,"name":"ios_rpan_broadcaster_award_presence","variant":"enabled","version":"3"},"ios_rpan_chat_award_presence":{"id":774,"name":"ios_rpan_chat_award_presence","variant":"enabled","version":"3"},"ios_share_message_action":{"id":451,"name":"ios_share_message_action","variant":"link","version":"10"},"ios_subreddit_mentions_v2":{"id":381,"name":"ios_subreddit_mentions_v

In [39]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path=r'/home/voileravi/chromedriver')
driver.get(url)

In [45]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse 
yesterday = datetime.today() - timedelta(days=1)

links = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]')

for a in links:
    if a.text.startswith('Daily Discussion Thread'):
        date = "".join(a.text.split(' ')[4:6])
        parsed = parse(date) 
        if parse(str(yesterday.date())) == parsed:
            link = a.find_element_by_xpath('../..').get_attribute('href')
    if a.text.startswith('Weekend'):
        weekend_date = a.text.split(' ')
        parsed_date = weekend_date[-3] + ' ' + weekend_date[-2] + weekend_date[-1] 
        parsed = parse(parsed_date) 
        saturday = weekend_date[-3] + ' ' +  str(int(weekend_date[-2].replace(',','')) - 1) + ' ' + weekend_date[-1] 
        
        if parse(str(yesterday.date())) == parsed: 
            link = a.find_element_by_xpath('../..').get_attribute('href')
        elif parse(str(yesterday.date())) == parse(str(saturday)):
            link = a.find_element_by_xpath('../..').get_attribute('href') 

stock_link = link.split('/')[-3]

InvalidSessionIdException: Message: invalid session id


In [41]:
html = requests.get(f'https://api.pushshift.io/reddit/submission/comment_ids/{stock_link}')
raw_comment_list = html.json()
driver.close() 

In [42]:
r = requests.get('https://finnhub.io/api/v1/stock/symbol?exchange=US&token=')
stocks = r.json()
stocks_list = []
for a in stocks:
    stocks_list.append(' ' + a['symbol'] + ' ')

In [43]:
html = requests.get(f'https://api.pushshift.io/reddit/comment/search?ids={comment_list}&fields=body&size=1000')

newcomments = html.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [36]:
stock_dict = Counter()
for a in newcomments['data']:
    for ticker in stocks_list:
        if ticker in a['body']:
            stock_dict[ticker]+=1


In [37]:
orig_list = np.array(raw_comment_list['data'])
comment_list = ",".join(orig_list[0:2000])
remove_me = slice(0,2000)
cleaned = np.delete(comment_list, remove_me)
i = 0
while i < len(cleaned):
    print(len(cleaned))
    cleaned = np.delete(cleaned, remove_me)
    new_comments_list = ",".join(cleaned[0:2000])
    newcomments = get_comments(new_comments_list)
    get_stock_list(newcomments,stock_dict)
stock = dict(stock_dict) 

In [27]:
data = list(sorted(stock.items(), key=lambda x: x[1], reverse=True))
#    data = list(zip((stock.keys()),sorted(stock.values())))
with open('stock.csv','w') as w:
    writer = csv.writer(w, lineterminator='\n')
    writer.writerow(['Stock','Number of Mentions'])
    for a in data:
        writer.writerow(a)